# Парсер основного Дата-фрейма с характеристиками всех смартфонов для построения модели
Цель тетради: запарсить данные с сайта ebay по характеристикам товаров.
В резульате должны получиться 2 дадафрейма:
* Sales содержит названия товаров, а также статистику продаж (цены, общее количество продаж, общая стоимость продаж, ссылки на подробные характеристики.
* Characteristics содержит данные о характеристиках товаров (Их много, не перечесть)

Данные надо откуда-то взять.

За парсинг датасета из любого источника на ваш вкус вы получаете 20 баллов
За сбор датасета из разных источников (несколько готовых датасетов, но собранных из разных источников и объединенные общей логикой) вы получаете 10 баллов
За просто готовый датасет – 5 баллов
Если вы взяли готовый датасет и обогатили его, спарсив что-то, вы получаете 20 баллов. Тут важно спарсить что-то полезное, обогащающее датасет. За парсинг book to scrape ради галочки, вы баллов не получите. Не забудьте вставить ссылки на всё, что используете в итоговый блокнот и подробно описать свои действия.

Рекомендация к парсингу: Данное условие не является строгим и обязательным, а скорее рекомендация для вас. Если вы решили парсить данные, рекомендуем убедиться, что вы сможете собрать хотя бы 4-5 числовых признаков и несколько категориальных для вашей модели (если вы собираетесь что-то предсказывать). При этом также рекомендуем собрать хотя бы 1000 наблюдений для модели.

Импортируем библиотеки.

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from random import randint
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore', message='The frame.append method is deprecated and will be removed from pandas in a future version.')

___

# Парсим первый датафрейм - Sales

In [3]:
# настраиваем драйвер
options = Options()
options.headless = True  # hide GUI
options.add_argument("--window-size=1920,1080")  # set window size to native GUI size
options.add_argument("start-maximized")  # ensure window is full-screen

# Настраиваем браузер, чтобы он не грузил Java и картинки
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options, chrome_options=chrome_options)

# Делаем запрос
driver.get("https://www.ebay.com/sh/research?marketplace=EBAY-US&keywords=Smartphones&dayRange=730&endDate=1684051559220&startDate=1620979559220&categoryId=9355&offset=0&limit=50&tabName=SOLD&tz=Europe%2FMoscow")

#Ждём загрузку
time.sleep(5)

C:\Users\druny\AppData\Local\Temp\ipykernel_19872\60259596.py:3: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True  # hide GUI
C:\Users\druny\AppData\Local\Temp\ipykernel_19872\60259596.py:9: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(options=options, chrome_options=chrome_options)


Тут мы ручками авторизируемся и заходим

Парсим названия и числовые характеристики (Цены, продажи)

In [ ]:
df = pd.DataFrame()

In [5]:
def table_parser(soup: BeautifulSoup):
    for product in soup.find_all('tr', class_='research-table-row'):
        title = product.find('span', attrs={'data-item-id': True}).text
        price = product.find('td', class_='research-table-row__item research-table-row__avgSoldPrice').div.find_all('div')[0].text.strip()
        shipping = product.find('td', class_='research-table-row__item research-table-row__avgShippingCost').div.find_all('div')[0].text.strip()
        sales_count = product.find('td', class_='research-table-row__item research-table-row__totalSoldCount').div.find_all('div')[0].text.strip()
        sales_value = product.find('td', class_='research-table-row__item research-table-row__totalSalesValue').div.find_all('div')[0].text.strip()
        try:
            link = product.find('a', class_='research-table-row__link-row-anchor', attrs={'href': True})['href']
        except:
            link = None

        data = {"Title": title, "Price": price, "Shipping_cost": shipping, "Sales_Count": sales_count, "Sales_Value": sales_value, "Link": link}

        global df
        df = df.append(data, ignore_index=True)

In [6]:
for _ in tqdm(range(100)): #Решено спарсить 100 страниц
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table_parser(soup)
    next_button = driver.find_element(By.XPATH, '//button[@class="pagination__next"]')
    next_button.click()
    time.sleep(randint(5, 15))
    pass

100%|██████████| 100/100 [18:25<00:00, 11.05s/it]


In [11]:
df.to_csv("Sales.tsv", sep='\t', index=False)

____
# Парсим характеристики - request

In [2]:
df=pd.read_csv("Sales.tsv", delimiter = "\t")

In [3]:
Links = df.Link.tolist()
Links[0:10]

['https://www.ebay.com/itm/254604777645?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/254187678666?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/363183815277?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/382605209867?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/382446518910?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/401763851775?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/362534145919?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/254733458977?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/254449547850?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/362643917935?nordt=true&orig_cvip=true&rt=nc',
 nan,
 'https://www.ebay.com/itm/362643928499?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/254343826760?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/383240884447?nordt=true&orig_cvip=true&rt=nc',
 'https://www.ebay.com/itm/40327

In [4]:
while np.nan in Links:
    Links.remove(np.nan)
None in Links

False

In [5]:
len(Links)

2924

In [8]:
import time
import random

characteristics_df = pd.DataFrame()
undone = []

# Задержка перед каждым запросом
def delay():
    time.sleep(random.uniform(1, 5))  # Измените значения задержки по своему усмотрению

def req_parser(url):
    global characteristics_df, undone

    try:
        # Отправляем GET-запрос и получаем ответ
        response = requests.get(url)

        # Получаем HTML-код страницы
        html = response.text

        # Создаем объект BeautifulSoup
        soup = BeautifulSoup(html, 'html.parser')

        # Находим родительский элемент
        parent_element = soup.find('div', class_='ux-layout-section--features')

        # Initialize an empty dictionary to store the characteristics
        characteristics = {"Link": url}
        done = []

        # Find all the characteristic rows
        rows = parent_element.find_all('div', class_='ux-layout-section-evo__row')

        # Iterate over each row and extract the characteristic name and value
        for row in rows:
            name_element = row.find_all('div', class_='ux-labels-values__labels')
            value_element = row.find_all('div', class_='ux-labels-values__values-content')

            for name, value in zip(name_element, value_element):
                if name and value:
                    name_text = name.get_text(strip=True)
                    value_text = value.get_text(strip=True)
                    if name_text not in done and value_text not in done:
                        characteristics[name_text] = value_text
                        done.append(name_text)
                        done.append(value_text)

        characteristics_df = characteristics_df.append(characteristics, ignore_index=True)

        # Задержка перед следующим запросом
        delay()

    except:
        print("Нащальника, не получилось:", url)
        undone.append(url)


for url in tqdm(Links):
    req_parser(url)

characteristics_df.to_csv("characteristics.tsv", sep='\t', index=False)

 19%|█▊        | 546/2924 [43:47<2:12:32,  3.34s/it]

Нащальника, не получилось: https://www.ebay.com/itm/254434966762?nordt=true&orig_cvip=true&rt=nc


 36%|███▌      | 1044/2924 [1:21:46<1:55:28,  3.69s/it]

Нащальника, не получилось: https://www.ebay.com/itm/143883489459?nordt=true&orig_cvip=true&rt=nc


 41%|████      | 1205/2924 [1:33:45<1:26:48,  3.03s/it]

...

 66%|██████▌   | 1932/2924 [2:29:34<1:01:07,  3.70s/it]

Нащальника, не получилось: https://www.ebay.com/itm/125489567873?nordt=true&orig_cvip=true&rt=nc


 69%|██████▉   | 2021/2924 [2:36:38<1:00:55,  4.05s/it]

Нащальника, не получилось: https://www.ebay.com/itm/353022115096?nordt=true&orig_cvip=true&rt=nc


100%|██████████| 2924/2924 [3:48:11<00:00,  4.68s/it]  
